# ClickUp Langchain Toolkit

In [1]:
%reload_ext autoreload
%autoreload 2
from datetime import datetime

from langchain.agents.agent_toolkits.clickup.toolkit import ClickupToolkit

from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.utilities.clickup import ClickupAPIWrapper


# Init

## Get Authenticated
1. Create a [ClickUp App](https://help.clickup.com/hc/en-us/articles/6303422883095-Create-your-own-app-with-the-ClickUp-API)
2. Follow [these steps](https://clickup.com/api/developer-portal/authentication/) to get your `client_id` and `client_secret`.
    - *Suggestion: use `https://google.com` as the redirect_uri. This is what we assume in the defaults for this toolkit.*
3. Copy/paste them and run the next cell to get your `code`


In [48]:
# Copilot Sandbox
oauth_client_id = "ABC..."
oauth_client_secret = "123..."
redirect_uri = "https://google.com"

print('Click this link, select your workspace, click `Connect Workspace`')
print(ClickupAPIWrapper.get_access_code_url(oauth_client_id, redirect_uri))

Click this link, select your workspace, click `Connect Workspace`
https://app.clickup.com/api?client_id=ABC...&redirect_uri=https://google.com


The url should change to something like this https://www.google.com/?code=THISISMYCODERIGHTHERE.

Next, copy/paste the `CODE` (THISISMYCODERIGHTHERE) generated in the URL in the cell below.


In [4]:
code = "THISISMYCODERIGHTHERE"

### Get Access Token
Then, use the code below to get your `access_token`.

*Important*: Each code is a one time code that will expire after use. The `access_token` can be used for a period of time. Make sure to copy paste the `access_token` once you get it!

In [5]:
access_token = ClickupAPIWrapper.get_access_token(oauth_client_id, oauth_client_secret, code)

if access_token is not None:
    print('Copy/paste this code, into the next cell so you can reuse it!')
    print(access_token)

Error: {'err': 'Code already used', 'ECODE': 'OAUTH_014'}
You already used this code once. Go back a step and generate a new code.
Our best guess for the url to get a new code is:
https://app.clickup.com/api?client_id=B5D61F8EVO04PR0JX0U73984LLS9GI6P&redirect_uri=https://google.com


## Create Toolkit

In [3]:
# Set your access token here
access_token = '12345678_myaccesstokengoeshere123'
access_token = '81928627_c009bf122ccf36ec3ba3e0ef748b07042c5e4217260042004a5934540cb61527'

# Init toolkit
clickup_api_wrapper = ClickupAPIWrapper(access_token=access_token)
toolkit = ClickupToolkit.from_clickup_api_wrapper(clickup_api_wrapper)
print(f'Found team_id: {clickup_api_wrapper.team_id}.\nMost request require the team id, so we store it for you in the toolkit, we assume the first team in your list is the one you want. \nNote: If you know this is the wrong ID, you can pass it at initialization.')

Found team_id: 9011010153.
Most request require the team id, so we store it for you in the toolkit, we assume the first team in your list is the one you want. 
Note: If you know this is the wrong ID, you can pass it at initialization.


## Create Agent

In [5]:
llm = OpenAI(temperature=0, openai_api_key="")

agent = initialize_agent(
    toolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Run

In [6]:
# helper function for demo
def print_and_run(command):
    print('\033[94m$ COMMAND\033[0m')
    print(command)
    print('\n\033[94m$ AGENT\033[0m')
    response = agent.run(command)
    print(''.join(['-']*80))
    return response
    

## Navigation
You can get the teams, folder and spaces your user has access to

In [7]:
print_and_run("Get all the teams that the user is authorized to access")
print_and_run("Get all the spaces available to the team")
print_and_run("Get all the folders for the team")

$ COMMAND
Get all the teams that the user is authorized to access

$ AGENT


> Entering new AgentExecutor chain...
 I need to use the Get Teams tool
Action: Get Teams
Action Input: No necessary request parameters
Observation: {'teams': [{'id': '9011010153', 'name': 'Task Copilot Sandbox Workspace 1', 'members': [{'id': 61681706, 'username': 'Aiswarya ', 'email': 'asankar@clickup.com', 'initials': 'A'}, {'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'email': 'rlentini@clickup.com', 'initials': 'RL'}]}]}
Thought: I now know the teams the user is authorized to access
Final Answer: The user is authorized to access the team 'Task Copilot Sandbox Workspace 1'.

> Finished chain.
--------------------------------------------------------------------------------
$ COMMAND
Get all the spaces available to the team

$ AGENT


> Entering new AgentExecutor chain...
 I need to use the API to get the spaces
Action: Get Teams
Action Input: No necessary request parameters
Observation: {'teams':

'The folders in the team are listed in the observation.'

## Task Operations
You can get, ask question about tasks and update them

In [32]:
task_id = '8685mb5fn'

#### Basic attirbute getting and updating

In [33]:
# We can get a task to inspect it's contents
print_and_run(f"Get task with id {task_id}")

# We can get a specific attribute from a task
previous_description = print_and_run(f"What is the description of task with id {task_id}")

# We can even update it!
print_and_run(f"For task with id {task_id}, change the description to 'A cool task descriptiont changed by AI!'")
print_and_run(f"What is the description of task with id {task_id}")

# Undo what we did
print_and_run(f"For task with id {task_id}, change the description to '{previous_description}'")

$ COMMAND
Get task with id 8685mb5fn

$ AGENT


> Entering new AgentExecutor chain...
 I need to use the Get task tool
Action: Get task
Action Input: {"task_id": "8685mb5fn"}
Observation: {'id': '8685mb5fn', 'name': 'dummy task 1', 'text_content': 'An old, boring task description', 'description': 'An old, boring task description', 'status': 'to do', 'creator_id': 81928627, 'creator_username': 'Rodrigo Ceballos Lentini', 'creator_email': 'rlentini@clickup.com', 'assignees': [], 'watcher_username': 'Rodrigo Ceballos Lentini', 'watcher_email': 'rlentini@clickup.com', 'priority': 'high', 'due_date': '1694764800000', 'start_date': None, 'points': None, 'team_id': '9011010153', 'project_id': '90110331875'}
Thought: I now know the task details
Final Answer: The task with id 8685mb5fn has the following details: {'id': '8685mb5fn', 'name': 'dummy task 1', 'text_content': 'An old, boring task description', 'description': 'An old, boring task description', 'status': 'to do', 'creator_id': 8192862

"The description of task 8685mb5fn has been updated to 'An old, boring task description'."

In [35]:
print_and_run("Change the descrition task 8685mj6cd to 'Look ma no hands'")

$ COMMAND
Change the descrition task 8685mj6cd to 'Look ma no hands'

$ AGENT


> Entering new AgentExecutor chain...
 I need to update the description of a task
Action: Update task
Action Input: {"task_id": "8685mj6cd", "attribute_name": "description", "value": "Look ma no hands"}
Observation: <Response [200]>
Thought: The task description has been successfully updated
Final Answer: The description of task 8685mj6cd has been changed to 'Look ma no hands'.

> Finished chain.
--------------------------------------------------------------------------------


"The description of task 8685mj6cd has been changed to 'Look ma no hands'."

#### Advanced Attributes (Assignees)
You can query and update almost every thing about a task!

In [36]:
user_id = 81928627

In [37]:
print_and_run(f"What are the assignees of task id {task_id}?")
print_and_run(f"Remove user {user_id} from the assignees of task id {task_id}")
print_and_run(f"What are the assignees of task id {task_id}?")
print_and_run(f"Add user {user_id} from the assignees of task id {task_id}")


$ COMMAND
What are the assignees of task id 8685mb5fn?

$ AGENT


> Entering new AgentExecutor chain...
 I need to get the assignees of a task
Action: Get task attribute
Action Input: {"task_id": "8685mb5fn", "attribute_name": "assignee"}
Observation: Error: attribute_name = assignee was not found in task keys dict_keys(['id', 'name', 'text_content', 'description', 'status', 'creator_id', 'creator_username', 'creator_email', 'assignees', 'watcher_username', 'watcher_email', 'priority', 'due_date', 'start_date', 'points', 'team_id', 'project_id']). Please call again with one of the key names.
Thought: I need to get the assignees of a task
Action: Get task attribute
Action Input: {"task_id": "8685mb5fn", "attribute_name": "assignees"}
Observation: []
Thought: I now know the final answer
Final Answer: There are no assignees for task id 8685mb5fn.

> Finished chain.
--------------------------------------------------------------------------------
$ COMMAND
Remove user 81928627 from the assi

'User 81928627 has been removed from the assignees of task id 8685mb5fn.'

## Creation
You can create tasks, lists and folders

In [27]:
time_str = datetime.now().strftime("%d/%m/%Y-%H:%M:%S")
print_and_run(f"Create a task called 'Test Task - {time_str}' with description 'This is a Test'")

$ COMMAND
Create a task called 'Test Task - 18/09/2023-10:31:22' with description 'This is a Test'

$ AGENT


> Entering new AgentExecutor chain...
 I need to use the Create Task tool
Action: Create Task
Action Input: {"name": "Test Task - 18/09/2023-10:31:22", "description": "This is a Test"}
Observation: {'id': '8685mw4wq', 'custom_id': None, 'name': 'Test Task - 18/09/2023-10:31:22', 'text_content': 'This is a Test', 'description': 'This is a Test', 'status': {'id': 'p90110061901_VlN8IJtk', 'status': 'to do', 'color': '#87909e', 'orderindex': 0, 'type': 'open'}, 'orderindex': '23.00000000000000000000000000000000', 'date_created': '1695047486396', 'date_updated': '1695047486396', 'date_closed': None, 'date_done': None, 'archived': False, 'creator': {'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'color': '#c51162', 'email': 'rlentini@clickup.com', 'profilePicture': None}, 'assignees': [], 'watchers': [{'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'color': '#c51162

"A task called 'Test Task - 18/09/2023-10:31:22' with description 'This is a Test' was successfully created."

In [29]:
time_str = datetime.now().strftime("%d/%m/%Y-%H:%M:%S")
print_and_run(f"Create a list called Test List - {time_str}")


$ COMMAND
Create a list called Test List - 18/09/2023-10:32:12

$ AGENT


> Entering new AgentExecutor chain...
 I need to create a list
Action: Create List
Action Input: {"name": "Test List - 18/09/2023-10:32:12"}
Observation: {'id': '901100774700', 'name': 'Test List - 18/09/2023-10:32:12', 'deleted': False, 'orderindex': 13, 'content': '', 'priority': None, 'assignee': None, 'due_date': None, 'start_date': None, 'folder': {'id': '90110336890', 'name': 'Test Folder', 'hidden': False, 'access': True}, 'space': {'id': '90110061901', 'name': 'Space', 'access': True}, 'inbound_address': 'a.t.901100774700.u-81928627.20b87d50-eece-4721-b487-9ca500338587@tasks.clickup.com', 'archived': False, 'override_statuses': False, 'statuses': [{'id': 'p90110061901_VlN8IJtk', 'status': 'to do', 'orderindex': 0, 'color': '#87909e', 'type': 'open'}, {'id': 'p90110061901_14GpYKnM', 'status': 'complete', 'orderindex': 1, 'color': '#6bc950', 'type': 'closed'}], 'permission_level': 'create'}
Thought: I now k

'The list "Test List - 18/09/2023-10:32:12" has been created with id 901100774700.'

In [30]:
time_str = datetime.now().strftime("%d/%m/%Y-%H:%M:%S")
print_and_run(f"Create a folder called 'Test Folder - {time_str}'")


$ COMMAND
Create a folder called 'Test Folder - 18/09/2023-10:32:51'

$ AGENT


> Entering new AgentExecutor chain...
 I need to use the Create Folder tool
Action: Create Folder
Action Input: {"name": "Test Folder - 18/09/2023-10:32:51"}
Observation: {'id': '90110348711', 'name': 'Test Folder - 18/09/2023-10:32:51', 'orderindex': 12, 'override_statuses': False, 'hidden': False, 'space': {'id': '90110061901', 'name': 'Space', 'access': True}, 'task_count': '0', 'archived': False, 'statuses': [], 'lists': [], 'permission_level': 'create'}
Thought: I have successfully created the folder
Final Answer: The folder 'Test Folder - 18/09/2023-10:32:51' has been successfully created.

> Finished chain.
--------------------------------------------------------------------------------


"The folder 'Test Folder - 18/09/2023-10:32:51' has been successfully created."

In [32]:
time_str = datetime.now().strftime("%d/%m/%Y-%H:%M:%S")
print_and_run(f"Create a list called 'Test List - {time_str}' with content My test list with high priority and status red")


$ COMMAND
Create a list called 'Test List - 18/09/2023-10:34:01' with content My test list with high priority and status red

$ AGENT


> Entering new AgentExecutor chain...
 I need to create a list with the given name, content, priority and status
Action: Create List
Action Input: {"name": "Test List - 18/09/2023-10:34:01", "content": "My test list", "priority": 2, "status": "red"}
Observation: {'id': '901100774746', 'name': 'Test List - 18/09/2023-10:34:01', 'deleted': False, 'orderindex': 15, 'content': '', 'status': {'status': 'red', 'color': '#e50000', 'hide_label': True}, 'priority': {'priority': 'high', 'color': '#ffcc00'}, 'assignee': None, 'due_date': None, 'start_date': None, 'folder': {'id': '90110336890', 'name': 'Test Folder', 'hidden': False, 'access': True}, 'space': {'id': '90110061901', 'name': 'Space', 'access': True}, 'inbound_address': 'a.t.901100774746.u-81928627.2ab87133-728e-4166-b2ae-423cc320df37@tasks.clickup.com', 'archived': False, 'override_statuses': False,

"The list 'Test List - 18/09/2023-10:34:01' with content 'My test list' with high priority and status red has been successfully created."

# Multi-Step Tasks

In [34]:
print_and_run("Figure out what user ID Rodrigo is, create a task called 'Rod's task', assign it to Rodrigo")

$ COMMAND
Figure out what user ID Rodrigo is, create a task called 'Rod's task', assign it to Rodrigo

$ AGENT


> Entering new AgentExecutor chain...
 I need to get the user ID of Rodrigo, create a task, and assign it to Rodrigo
Action: Get Teams
Action Input: No input necessary
Observation: {'teams': [{'id': '9011010153', 'name': 'Task Copilot Sandbox Workspace 1', 'members': [{'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'email': 'rlentini@clickup.com', 'initials': 'RL'}]}]}
Thought: I now have the user ID of Rodrigo
Action: Create Task
Action Input: {"name": "Rod's task", "assignees": [81928627]}

/Users/rodrigolentini/repos/langchain-clickup/libs/langchain/langchain/utilities/clickup.py:145: UserWarning: Error encountered while trying to parse <class 'langchain.utilities.clickup.Task'>: 'NoneType' object is not subscriptable
 Falling back to returning input data.
  warnings.warn(f'Error encountered while trying to parse {dataclass}: {e}\n Falling back to returning input data.')



Observation: {'id': '8685mw6dz', 'custom_id': None, 'name': "Rod's task", 'text_content': '', 'description': '', 'status': {'id': 'p90110061901_VlN8IJtk', 'status': 'to do', 'color': '#87909e', 'orderindex': 0, 'type': 'open'}, 'orderindex': '24.00000000000000000000000000000000', 'date_created': '1695047740939', 'date_updated': '1695047740939', 'date_closed': None, 'date_done': None, 'archived': False, 'creator': {'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'color': '#c51162', 'email': 'rlentini@clickup.com', 'profilePicture': None}, 'assignees': [{'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'color': '#c51162', 'initials': 'RL', 'email': 'rlentini@clickup.com', 'profilePicture': None}], 'watchers': [{'id': 81928627, 'username': 'Rodrigo Ceballos Lentini', 'color': '#c51162', 'initials': 'RL', 'email': 'rlentini@clickup.com', 'profilePicture': None}], 'checklists': [], 'tags': [], 'parent': None, 'priority': None, 'due_date': None, 'start_date': None, 'points': 

"Rodrigo's user ID is 81928627 and a task called 'Rod's task' has been created and assigned to him."